### Creating Model for Classifying Emotions

In [8]:
#Importing libraries

import numpy as np
from imutils import paths
import os

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [4]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

2023-02-15 19:47:56.610576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
imagePaths = list(paths.list_images('Reaction_Dataset'))

In [6]:
# Extracting Data and Label

data = []
labels = []

IMG_SIZE = 224
CHANNELS = 3

for imagePath in imagePaths:
  label = imagePath.split(os.path.sep)[-2]
  image = load_img(imagePath, target_size=(IMG_SIZE, IMG_SIZE))
  image = img_to_array(image)
  image = image/255
  

  data.append(image)
  labels.append(label)



data = np.array(data, dtype="float32")
labels = np.array(labels)

In [7]:
#Labels

np. unique(labels, return_counts=True)

(array(['.ipynb_checkpoints', 'Happy', 'Loss'], dtype='<U18'),
 array([   1, 2844, 1095]))

In [19]:
#Encoding Labels

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels)

In [20]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels)

In [21]:
from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(data, labels,	test_size=0.20, stratify=labels, random_state=42)

In [22]:
#ResNet50 is a big network with 50 layers
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input

In [23]:
feature_extractor_layer = ResNet50V2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(IMG_SIZE,IMG_SIZE,CHANNELS)))

2023-02-15 15:53:57.940759: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
feature_extractor_layer.trainable = False

In [26]:
#Model Creation

model = tf.keras.Sequential()
model.add(feature_extractor_layer)
model.add(layers.Flatten(name="flatten"))
model.add(layers.Dense(1024, activation='relu', name='hidden_layer'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax', name='output'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 hidden_layer (Dense)        (None, 1024)              102761472 
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 output (Dense)              (None, 2)                 2050      
                                                                 
Total params: 126,328,322
Trainable params: 102,763,522
Non-trainable params: 23,564,800
_________________________________________________________________


In [28]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  loss="categorical_crossentropy",
  metrics=["accuracy"])

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [30]:
history = model.fit(aug.flow(trainX, trainY),
	                  validation_data=(testX, testY),
	                  epochs=10)

Epoch 1/10
99/99 [==============================] - 262s 3s/step - loss: 1.1470 - accuracy: 0.6604 - val_loss: 0.6592 - val_accuracy: 0.7259
Epoch 2/10
99/99 [==============================] - 267s 3s/step - loss: 0.8030 - accuracy: 0.7080 - val_loss: 0.6331 - val_accuracy: 0.7538
Epoch 3/10
99/99 [==============================] - 265s 3s/step - loss: 0.6759 - accuracy: 0.7312 - val_loss: 0.5728 - val_accuracy: 0.7525
Epoch 4/10
99/99 [==============================] - 272s 3s/step - loss: 0.6399 - accuracy: 0.7274 - val_loss: 0.5541 - val_accuracy: 0.7475
Epoch 5/10
99/99 [==============================] - 280s 3s/step - loss: 0.5654 - accuracy: 0.7547 - val_loss: 0.5152 - val_accuracy: 0.7525
Epoch 6/10
99/99 [==============================] - 270s 3s/step - loss: 0.5403 - accuracy: 0.7544 - val_loss: 0.5227 - val_accuracy: 0.7437
Epoch 7/10
99/99 [==============================] - 272s 3s/step - loss: 0.5074 - accuracy: 0.7664 - val_loss: 0.4846 - val_accuracy: 0.7589
Epoch 8/10
99

In [31]:
from sklearn.metrics import classification_report


In [32]:
predIdxs = model.predict(testX)

25/25 [==============================] - 43s 2s/step


In [37]:
predIdxs[501]

array([0.04677653, 0.9532235 ], dtype=float32)

In [38]:
predIdxs = np.argmax(predIdxs, axis=1)

In [39]:
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=le.classes_))

              precision    recall  f1-score   support

       Happy       0.83      0.88      0.85       569
        Loss       0.62      0.52      0.57       219

    accuracy                           0.78       788
   macro avg       0.72      0.70      0.71       788
weighted avg       0.77      0.78      0.77       788



In [40]:
#Model for classifying moments

model.save('Model_Moments_CNN.h5', save_format="h5")

In [43]:
model = tf.keras.models.load_model('Model_Moments_CNN.h5')